In [5]:
import sqlite3
from sqlite3 import Error
import hashlib

# Database connection function
def create_connection(db_file):
    conn = None
    try:
        conn = sqlite3.connect(db_file)
    except Error as e:
        print(e)
    return conn

# Function to validate the email domain (only gmail or hotmail allowed)
def validate_email(email):
    if email.endswith('@gmail.com') or email.endswith('@hotmail.com'):
        return True
    else:
        print("Only Gmail and Hotmail emails are allowed.")
        return False

# Create Users Table
def create_user_table(conn):
    sql_create_user_table = """CREATE TABLE IF NOT EXISTS Users (
                                user_id INTEGER PRIMARY KEY,
                                email TEXT NOT NULL UNIQUE,
                                password TEXT NOT NULL,
                                role TEXT NOT NULL);"""
    try:
        c = conn.cursor()
        c.execute(sql_create_user_table)
    except Error as e:
        print(e)

# Create Events Table
def create_event_table(conn):
    sql_create_event_table = """CREATE TABLE IF NOT EXISTS Events (
                                event_id INTEGER PRIMARY KEY,
                                title TEXT NOT NULL,
                                description TEXT,
                                date TEXT,
                                location TEXT,
                                organizer_id INTEGER,
                                FOREIGN KEY (organizer_id) REFERENCES Users(user_id));"""
    try:
        c = conn.cursor()
        c.execute(sql_create_event_table)
    except Error as e:
        print(e)

# Create Enrollment Table
def create_enrollment_table(conn):
    sql_create_enrollment_table = """CREATE TABLE IF NOT EXISTS Enrollment (
                                      user_id INTEGER,
                                      event_id INTEGER,
                                      PRIMARY KEY (user_id, event_id),
                                      FOREIGN KEY (user_id) REFERENCES Users(user_id),
                                      FOREIGN KEY (event_id) REFERENCES Events(event_id));"""
    try:
        c = conn.cursor()
        c.execute(sql_create_enrollment_table)
    except Error as e:
        print(e)

# Create all necessary tables (User, Event, Enrollment)
def create_tables(conn):
    create_user_table(conn)  # Users Table
    create_event_table(conn)  # Events Table
    create_enrollment_table(conn)  # Enrollment Table


In [6]:
# Register a new user (email, password, role)
def register_user(conn):
    while True:
        email = input("Enter your email to register: ")
        if not validate_email(email):  # If the email is not valid
            print("Invalid email. Please enter a valid Gmail or Hotmail email.")
            continue  # Restart the process of email entry if it's invalid

        password = input("Enter your password: ")
        role = input("Enter your role (attendee/organizer): ")

        # Check if the email already exists in the database
        sql_check_email = "SELECT * FROM Users WHERE email = ?"
        cur = conn.cursor()
        cur.execute(sql_check_email, (email,))
        existing_user = cur.fetchone()

        if existing_user:
            print("This email is already registered. Please try a different one.")
        else:
            # If email is unique, proceed with registration
            hashed_password = hashlib.sha256(password.encode('utf-8')).hexdigest()  # SHA-256 hashing
            sql = "INSERT INTO Users (email, password, role) VALUES (?, ?, ?)"
            values = (email, hashed_password, role)
            try:
                cur.execute(sql, values)
                conn.commit()  # Commit the transaction to make changes persistent
                print("Registration successful!")
                break  # Exit the loop when registration is successful
            except sqlite3.OperationalError as e:
                print(f"Error: {e}")
                conn.rollback()  # Rollback any changes if an error occurs
                break  # Exit after an error (you can adjust this behavior)
            finally:
                # Close the cursor to ensure no lingering database operations
                cur.close()

# Login User (email, password)
def login_user(conn, email, password):
    if not validate_email(email):  # Validate the email before proceeding
        return None

    # Hash the entered password using SHA-256
    hashed_password = hashlib.sha256(password.encode('utf-8')).hexdigest()

    sql = "SELECT * FROM Users WHERE email = ?"
    values = (email,)
    cur = conn.cursor()
    cur.execute(sql, values)
    user = cur.fetchone()

    if user:
        stored_hashed_password = user[2]  # Stored hashed password in the database
        if hashed_password == stored_hashed_password:  # Compare password with stored hash
            print(f"Welcome back, {user[1]}!")  # user[1] is the email
            return user  # Returns user details
        else:
            print("Invalid password.")
            return None
    else:
        print("Email not found in database.")
        return None


In [7]:
# View events
def view_events(conn):
    sql = "SELECT event_id, title, description, date, location FROM Events"
    cur = conn.cursor()
    cur.execute(sql)
    events = cur.fetchall()
    print("Upcoming Events:")
    for event in events:
        print(f"ID: {event[0]} - Title: {event[1]}, Date: {event[3]}, Location: {event[4]}")

# Enroll in an event (for Attendee)
def enroll_in_event(conn, user_id, event_id):
    sql = "INSERT INTO Enrollment (user_id, event_id) VALUES (?, ?)"
    values = (user_id, event_id)
    cur = conn.cursor()
    cur.execute(sql, values)
    conn.commit()
    print("You have successfully enrolled in the event!")

# Create an event (for Organizer)
def create_event(conn, title, description, date, location, organizer_id):
    sql = """INSERT INTO Events (title, description, date, location, organizer_id)
             VALUES (?, ?, ?, ?, ?)"""
    values = (title, description, date, location, organizer_id)
    cur = conn.cursor()
    cur.execute(sql, values)
    conn.commit()
    print("Event created successfully!")

# After login or registration: Check the role and direct accordingly
def after_login_or_register(conn, user):
    user_id = user[0]  # user_id (first column in Users table)
    role = user[3]  # user role

    if role == 'attendee':
        # Attendee can view and enroll in events
        while True:
            print("1. View Events")
            print("2. Enroll in Event")
            print("3. Exit")
            choice = input("Enter your choice: ")

            if choice == '1':
                view_events(conn)
            elif choice == '2':
                event_id = int(input("Enter the event ID you want to enroll in: "))
                enroll_in_event(conn, user_id, event_id)
            elif choice == '3':
                break
            else:
                print("Invalid choice.")
    elif role == 'organizer':
        # Organizer can create events
        while True:
            print("1. Create Event")
            print("2. Exit")
            choice = input("Enter your choice: ")

            if choice == '1':
                title = input("Enter event title: ")
                description = input("Enter event description: ")
                date = input("Enter event date (YYYY-MM-DD): ")
                location = input("Enter event location: ")
                create_event(conn, title, description, date, location, user_id)
            elif choice == '2':
                break
            else:
                print("Invalid choice.")


In [ ]:
# Main function to run the program
def main():
    database = r"DatabaseProject.db"
    conn = create_connection(database)

    if conn is not None:
        create_tables(conn)  # Create necessary tables

        while True:
            choice = input("Do you want to [L]ogin or [R]egister? (L/R): ").strip().lower()

            if choice == 'r':
                register_user(conn)  # Call the modified register_user function
                break  # Exit after successful registration

            elif choice == 'l':
                email = input("Enter your email to login: ")
                password = input("Enter your password: ")
                user = login_user(conn, email, password)

                if user:
                    after_login_or_register(conn, user)  # Start the command interface after successful login
                    break
            else:
                print("Invalid choice. Exiting.")
                conn.close()  # Close the connection properly before exiting
                break

        conn.close()  # Ensure the connection is closed after all operations are done

if __name__ == '__main__':
    main()
